In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#Create input data which reads data from MNIST datasets. Perform on hot encoding to define the digit
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#Using interactive session makes it to default session so we don not have to use sess
sess = tf.InteractiveSession()

In [4]:
#Define placeholders from MNISt data
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [5]:
#change MNIST data from a list of values to 28px X 28px X 1 grayscale cube
# which the convolution NN can use
x_image = tf.reshape(x, [-1,28,28,1], name="x_image")

In [6]:
#Define helper functions to define bias and weights variables, convolution, and pooling layer
#we are using relu as our activation function. these must be utilized to a small positive number
# and with some noise so that we dont end up going to zero when comparing diffs

def weight_variables(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variables(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#Convolution and pooling - we do convolution and then pooling to control overfitting
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [7]:
#define layers in NN

#1st convolution layer
#32 features of 5x5 patch of image
W_conv1 = weight_variables([5,5,1,32])
b_conv1 = bias_variables([32])
#Do convolution on images, add bias and push through RELU activation
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# take results and run through max_pool
h_pool1 = max_pool_2x2(h_conv1) #reduced to 14x14 image

In [8]:
#2st convolution layer
#Process the 32 features from convolution1 layer, in of 5x5 patch. Return 64 features weights and biases
W_conv2 = weight_variables([5,5,32,64])
b_conv2 = bias_variables([64])
#Do convolution on output of 1st convolution layer. pool results
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
# take results and run through max_pool
h_pool2 = max_pool_2x2(h_conv2) #reduced to 7x7 image

In [9]:
#Fully connected layer
W_fc1 = weight_variables([7*7*64,1024])
b_fc1 = bias_variables([1024])

In [10]:
# connect output of pooling layer 2 as input to fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

In [11]:
# dropout some nuerons to reduce overfitting 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
#Readout layer
W_fc2 = weight_variables([1024,10])
b_fc2 = bias_variables([10])

In [13]:
#Define model
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [14]:
#Loss measurement
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

In [15]:
#loss optimzation
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [16]:
#What is correct
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
#how accurate is it
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
#initialize all the variables
sess.run(tf.global_variables_initializer())

In [18]:
#train the model
import time

In [19]:
#define nnumber of steps and how often we display progress
num_steps = 3000
display_every = 100

In [20]:
#Start timer
start_time = time.time()
end_time = time.time()

In [21]:
for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})
    
    #periodic status display
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
        end_time = time.time()
        print("step {0}, elapsed_time {1:.2f} seconds, training_accuracy {2:.3f}".format(i, end_time - start_time, train_accuracy * 100.0))

step 0, elapsed_time 0.61 seconds, training_accuracy 10.000
step 100, elapsed_time 35.26 seconds, training_accuracy 86.000
step 200, elapsed_time 70.14 seconds, training_accuracy 92.000
step 300, elapsed_time 105.17 seconds, training_accuracy 90.000
step 400, elapsed_time 140.21 seconds, training_accuracy 98.000
step 500, elapsed_time 174.79 seconds, training_accuracy 98.000
step 600, elapsed_time 212.28 seconds, training_accuracy 92.000
step 700, elapsed_time 250.93 seconds, training_accuracy 96.000
step 800, elapsed_time 288.68 seconds, training_accuracy 100.000
step 900, elapsed_time 353.04 seconds, training_accuracy 94.000
step 1000, elapsed_time 399.71 seconds, training_accuracy 98.000
step 1100, elapsed_time 433.86 seconds, training_accuracy 94.000
step 1200, elapsed_time 467.62 seconds, training_accuracy 98.000
step 1300, elapsed_time 501.34 seconds, training_accuracy 98.000
step 1400, elapsed_time 535.19 seconds, training_accuracy 100.000
step 1500, elapsed_time 568.82 seconds,

In [22]:
#Display summary
#Time to train
end_time = time.time()
print("total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))

total training time for 3000 batches: 1043.56 seconds


In [23]:
#Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0})*100.0))

Test accuracy 98.040%


In [24]:
sess.close()